<br><br><br><br><br><h1 style="font-size:4em;color:#2467C0">Welcome to Week 3</h1><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">This document provides a running example of completing the Week 3 assignment : </p>
<ul class="simple">
<li style="line-height:31px;">A shorter version with fewer comments is available as script: sparkMLlibClustering.py</li>
<li style="line-height:31px;">To run these commands in Cloudera VM: first run the setup script: setupWeek3.sh</li>
<li style="line-height:31px;">You can then copy paste these commands in pySpark. </li>
<li style="line-height:31px;">To open pySpark, refer to : <a class="reference external" href="https://www.coursera.org/learn/machinelearningwithbigdata/supplement/GTFQ0/slides-module-2-lesson-3">Week 2</a> and <a class="reference external" href="https://www.coursera.org/learn/machinelearningwithbigdata/supplement/RH1zz/download-lesson-2-slides-spark-mllib-clustering">Week 4</a> of the Machine Learning course</li>
<li style="line-height:31px;">Note that your dataset may be different from what is used here, so your results may not match with those shown here</li>
</ul></div>

In [194]:
import pandas as pd
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from math import sqrt

<br><br>
<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">
<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 1: Attribute Selection</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Import Data</h1><br><br> 


<p style="line-height:31px;">First let us read the contents of the file ad-clicks.csv. The following commands read in the CSV file in a table format and removes any extra whitespaces. So, if the CSV contained ' userid  ' it becomes 'userid'. <br><br>


Note that you must change the path to ad-clicks.csv to the location on your machine, if you want to run this command on your machine.
</p>

</div>

<br><br><br><br>

In [44]:
adclicksDF = pd.read_csv('./ad-clicks.csv')
adclicksDF = adclicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers

In [73]:
gameclicksDF = pd.read_csv('./game-clicks.csv')
gameclicksDF = gameclicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers

In [127]:
playTimeDF = pd.read_csv('./user-session.csv')
playTimeDF = playTimeDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of adclicksDF:</p>

</div>

<br><br><br><br>

In [45]:
adclicksDF.head(n=5)

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing


In [74]:
gameclicksDF.head(n=5)

,timestamp,clickId,userId,userSessionId,isHit,teamId,teamLevel
0,2016-05-26 15:06:55,105,1038,5916,0,25,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1


In [128]:
playTimeDF.head(n=10)

,timestamp,userSessionId,userId,teamId,assignmentId,sessionType,teamLevel,platformType
0,2016-05-26 14:47:30,5675,170,90,5423,start,1,iphone
1,2016-05-27 00:06:54,5675,170,90,5423,end,1,iphone
2,2016-05-26 14:43:09,5795,835,39,5300,start,1,android
3,2016-05-27 08:06:54,5795,835,39,5300,end,1,android
4,2016-05-26 14:47:20,5833,624,2,5529,start,1,iphone
5,2016-05-27 23:06:54,5833,624,2,5529,end,1,iphone
6,2016-05-26 14:37:33,5877,514,2,5528,start,1,windows
7,2016-05-27 23:06:54,5877,514,2,5528,end,1,windows
8,2016-05-26 14:36:55,5902,2103,2,5527,start,1,iphone
9,2016-05-27 23:06:54,5902,2103,2,5527,end,1,iphone


<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Next, We are going to add an extra column to the ad-clicks table and make it equal to 1. We do so to record the fact that each ROW is 1 ad-click. 
You will see how this will become useful when we sum up this column to find how many ads
did a user click.</p>

</div>

<br><br><br><br>

In [46]:
adclicksDF['adCount'] = 1

In [136]:
playTimeDF['sessionCount']=0.5

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of adclicksDF and see if 
a new column has been added:</p>

</div>

<br><br><br><br>

In [47]:
adclicksDF.head(n=5)

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory,adCount
0,2016-05-26 15:13:22,5974,5809,27,611,2,electronics,1
1,2016-05-26 15:17:24,5976,5705,18,1874,21,movies,1
2,2016-05-26 15:22:52,5978,5791,53,2139,25,computers,1
3,2016-05-26 15:22:57,5973,5756,63,212,10,fashion,1
4,2016-05-26 15:22:58,5980,5920,9,1027,20,clothing,1


In [137]:
playTimeDF.head(n=5)

,timestamp,userSessionId,userId,teamId,assignmentId,sessionType,teamLevel,platformType,sessionCount
0,2016-05-26 14:47:30,5675,170,90,5423,start,1,iphone,0.5
1,2016-05-27 00:06:54,5675,170,90,5423,end,1,iphone,0.5
2,2016-05-26 14:43:09,5795,835,39,5300,start,1,android,0.5
3,2016-05-27 08:06:54,5795,835,39,5300,end,1,android,0.5
4,2016-05-26 14:47:20,5833,624,2,5529,start,1,iphone,0.5


<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Next, let us read the contents of the file buy-clicks.csv. As before, the following commands read in the CSV file in a table format and removes any extra whitespaces. So, if the CSV contained ' userid  ' it becomes 'userid'. <br><br>


Note that you must change the path to buy-clicks.csv to the location on your machine, if you want to run this command on your machine.
</p>

</div>

<br><br><br><br>

In [67]:
buyclicksDF = pd.read_csv('./buy-clicks.csv')
buyclicksDF = buyclicksDF.rename(columns=lambda x: x.strip()) #removes whitespaces from headers

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of buyclicksDF:</p>

</div>

<br><br><br><br>

In [68]:
buyclicksDF.head(n=5)

,timestamp,txId,userSessionId,team,userId,buyId,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0


<br><br>

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Feature Selection</h1><br><br>


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">For this exercise, we can choose from buyclicksDF,  the 'price' of each app that a user purchases as an attribute that captures user's purchasing behavior. The following command selects 'userid' and 'price' and drops all other columns that we do not want to use at this stage.</p>


</div>

<br><br><br><br>

In [50]:
userPurchases = buyclicksDF[['userId','price']] #select only userid and price
userPurchases.head(n=5)

,userId,price
0,1300,3.0
1,868,10.0
2,819,20.0
3,121,3.0
4,2222,20.0


<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Similarly, from the adclicksDF,  we will use the 'adCount' as an attribute that captures user's inclination to click on ads. The following command selects 'userid' and 'adCount' and drops all other columns that we do not want to use at this stage.</p>


</div>

<br><br><br><br>

In [71]:
useradClicks = adclicksDF[['userId','adCount']]

In [72]:
useradClicks.head(n=5) #as we saw before, this line displays first five lines

,userId,adCount
0,611,1
1,1874,1
2,2139,1
3,212,1
4,1027,1


In [77]:
usergameHits = gameclicksDF[['userId','isHit']]

In [78]:
usergameHits.head(n=5)

,userId,isHit
0,1038,0
1,1099,0
2,899,0
3,2197,0
4,1362,0


In [138]:
userSessions = playTimeDF[['userId','sessionCount']]

In [139]:
userSessions.head(n=5)

,userId,sessionCount
0,170,0.5
1,170,0.5
2,835,0.5
3,835,0.5
4,624,0.5


<br><br>
<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">Step 2: Training Data Set Creation</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the first aggregate feature for clustering</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">From each of these single ad-clicks per row, we can now generate total ad clicks per user. Let's pick a user with userid = 3. To find out how many ads this user has clicked overall, we have to find each row that contains userid = 3, and report the total number of such rows.

The following commands sum the total number of ads per user and rename the columns to be called 'userid' and 'totalAdClicks'. <b> Note that you may not need to aggregate (e.g. sum over many rows) if you choose a different feature and your data set already provides the necessary information. </b> In the end, we want to get one row per user, if we are performing clustering over users.

</div>

<br><br><br><br>

In [53]:
adsPerUser = useradClicks.groupby('userId').sum()
adsPerUser = adsPerUser.reset_index()
adsPerUser.columns = ['userId', 'totalAdClicks'] #rename the columns

In [82]:

hitsPerUser = usergameHits.groupby('userId').sum()
hitsPerUser = hitsPerUser.reset_index()
hitsPerUser.columns = ['userId', 'totalHits'] #rename the columns



In [140]:

userSess = userSessions.groupby('userId').sum()
userSess = userSess.reset_index()
userSess.columns = ['userId', 'sessionCount'] #rename the columns



In [141]:
userSess.head(n=5)

,userId,sessionCount
0,0,4.0
1,1,7.0
2,2,5.0
3,6,1.0
4,8,2.0


In [83]:
hitsPerUser.head(n=5)

,userId,totalHits
0,0,143
1,1,96
2,2,22
3,6,16
4,8,38


<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Let us display the first 5 lines of 'adsPerUser' to see if there
is a column named 'totalAdClicks' containing total adclicks per user.</p>

</div>

<br><br><br><br>

In [81]:
adsPerUser.head(n=5)

,userId,totalAdClicks
0,1,44
1,8,10
2,9,37
3,10,19
4,12,46


<br><br>

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the second aggregate feature for clustering</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Similar to what we did for adclicks, here we find out how much money in total did each user spend on buying in-app purchases. As an example, let's pick a user with userid = 9. To find out the total money spent by this user, we have to find each row that contains userid = 9, and report the sum of the column'price' of each product they purchased.

The following commands sum the total money spent by each user and rename the columns to be called 'userid' and 'revenue'.
<br><br>

<p style="line-height:31px;"> <b> Note: </b> that you can also use other aggregates, such as sum of money spent on a specific ad category by a user or on a set of ad categories by each user, game clicks per hour by each user etc. You are free to use any mathematical operations on the fields provided in the CSV files when creating features. </p>


</div>

<br><br><br><br> 

In [55]:
revenuePerUser = userPurchases.groupby('userId').sum()
revenuePerUser = revenuePerUser.reset_index()
revenuePerUser.columns = ['userId', 'revenue'] #rename the columns

In [56]:
revenuePerUser.head(n=5)

,userId,revenue
0,1,21.0
1,8,53.0
2,9,80.0
3,10,11.0
4,12,215.0


<br><br>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Merge the two tables</h1><br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Lets see what we have so far. We have a table called revenuePerUser, where each row contains total money a user (with that 'userid') has spent. We also have another table called adsPerUser where each row contains total number of ads a user has clicked. We will use revenuePerUser and adsPerUser as features / attributes to capture our users' behavior.<br><br>

Let us combine these two attributes (features) so that each row contains both attributes per user. Let's merge these two tables to get one single table we can use for K-Means clustering.
</div>

<br><br><br><br> 

In [57]:
combinedDF = adsPerUser.merge(revenuePerUser, on='userId') #userId, adCount, price

In [85]:
combinedDF2 = combinedDF.merge(hitsPerUser, on='userId') #userId, hits

In [142]:
combinedDF3 = combinedDF2.merge(userSess, on='userId') #add sessionCount

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">


<p style="line-height:31px;">Let us display the first 5 lines of the merged table. <b> Note: Depending on what attributes you choose, you may not need to merge tables. You may get all your attributes from a single table. </b></p>

</div>

<br><br><br><br>

In [58]:
combinedDF.head(n=5) #display how the merged table looks

,userId,totalAdClicks,revenue
0,1,44,21.0
1,8,10,53.0
2,9,37,80.0
3,10,19,11.0
4,12,46,215.0


In [126]:
combinedDF2.head(n=5)

,userId,totalAdClicks,revenue,totalHits
0,1,44,21.0,96
1,8,10,53.0,38
2,9,37,80.0,62
3,10,19,11.0,340
4,12,46,215.0,92


<br><br>

<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Create the final training dataset</h1><br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Our training data set is almost ready. At this stage we can remove the 'userid' from each row, since 'userid' is a computer generated random number assigned to each user. It does not capture any behavioral aspect of a user. One way to drop the 'userid', is to select the other two columns. </p>

</div>

<br><br><br><br>

In [171]:
trainingDF = combinedDF3[['totalAdClicks','revenue','sessionCount']]
trainingDF.head(n=5)

,totalAdClicks,revenue,sessionCount
0,44,21.0,7.0
1,10,53.0,2.0
2,37,80.0,6.0
3,19,11.0,5.0
4,46,215.0,7.0


<br><br>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Display the dimensions of the training dataset</h1><br><br> 


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Display the dimension of the training data set. To display the dimensions of the trainingDF, simply add .shape as a suffix and hit enter.</p>

</div>

<br><br><br><br>


In [164]:
trainingDF.shape

(529, 3)

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">The following two commands convert the tables we created into a format that can be understood by the KMeans.train function. <br><br>

line[0] refers to the first column. line[1] refers to the second column. If you have more than 2 columns in your training table, modify this command by adding line[2], line[3], line[4] ...</p>

</div>

<br><br><br><br>


In [168]:
sqlContext = SQLContext(sc)
pDF = sqlContext.createDataFrame(trainingDF)
parsedData = pDF.rdd.map(lambda line: array([line[0], line[1], line[2]])) #totalAdClicks, revenue



<br>
<h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 3: Train to Create Cluster Centers</h1>
<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Train KMeans model</h1><br><br> 

<br><br><br><br>

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Here we are creating two clusters as denoted in the second argument.</p>

</div>

<br><br><br><br>


In [205]:
my_kmmodel = KMeans.train(parsedData, 3, maxIterations=10, initializationMode="random")

In [206]:
def error(point):
    center = my_kmmodel.centers[my_kmmodel.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))


In [207]:

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)

print("Within Set Sum of Squared Error = " + str(WSSSE))


Within Set Sum of Squared Error = 10023.577061352767


<br><br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Display the centers of two clusters formed</h1><br><br> 
One method for choosing the number of initial clusters is to try several different numbers of initial clusters, calculate the "Within-Cluster Sum of Squared Error" or "WSSE" for each and plot the WSSE for each value of the initial number of clusters. Where there is an elbow in the plot is the value of the initial number of clusters that should be used. This method is from the k-Means Clustering method for cluster analysis.
<br><br>

In [170]:
print(my_kmmodel.centers)

[array([  40.203125,  131.21875 ,    6.078125]), array([ 33.93333333,  52.71515152,   5.01818182]), array([ 25.77      ,  13.59      ,   3.83666667])]


<br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Step 4: Recommend Actions</h1>
<br><h1 style="font-family: Arial; font-size:1.5em;color:#2462C0">Analyze the cluster centers
</h1>

<br><br> 

<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Each array denotes the center for a cluster:<br><br>
One Cluster is centered at   ... array([ 29.43211679,  24.21021898])<br>
Other Cluster is centered at   ... array([  42.05442177,  113.02040816])</p>

<br><br>

<p style="line-height:31px;"> First number (field1) in each array refers to number of ad-clicks and the second number (field2) is the revenue per user.

Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads.

Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff. 

</p><br><br>

<p style="line-height:31px;">In one cluster, in general, players click on ads much more often (~1.4 times) and spend more money (~4.7 times) on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.</p>

<br><br>
<p style="line-height:31px;"> <b> Note: </b>  This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the  centers. The answer to this question will depend on the features you have chosen. <br><br> Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea. </p>

</div>
